Step 1: download the data from eotdl

In [ ]:
import os
import zipfile
import eotdl
from eotdl.datasets import download_dataset

download_dataset("EuroSAT", version=1, path="data", force=True)

os.makedirs("data/EuroSAT", exist_ok=True)

with zipfile.ZipFile("data/EuroSAT/v1/EuroSAT.zip", 'r') as zip_ref:
    zip_ref.extractall("data/EuroSAT")

Step 2: Make a random selection of labelled data for which we want to extract EO data, from the images we can readily extract their geospatial bounding box. We append this information into a dataframe which will lateron be used to orchestrate the EO data extraction.

In [16]:
from dataframe_utils import *

# Constants
src_dir = r"C:\Git_projects\eotdl_wip\data\EuroSAT\ds\images\remote_sensing"
num_files = 100 #number of images we wish to process

start_date = "2020-01-01"
nb_months = 3

distance_m = 320  # Buffer distance in meters
resolution = 20.0  # Grid resolution in meters

tif_files = get_tif_files(src_dir)
selected_files = random.sample(tif_files, num_files)

base_df = generate_geodataframe_per_utm(selected_files, start_date, nb_months, distance_m, resolution)
split_df = process_split_jobs(base_df, 10)
job_df = create_job_dataframe_s2(split_df)


c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["centroid"] = polygons.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:84: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_grid["geometry"] = s2_grid.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before t

Subselection for quick testing

In [17]:
# TODO; remove: for testing
job_df = job_df[job_df['feature_count'] > 1][0:2]
job_df

,temporal_extent,geometry,s2_tile,h3index,crs,feature_count
4,"[2020-01-01, 2020-04-01]","{""type"": ""FeatureCollection"", ""features"": [{""i...",33TXF,831e88fffffffff,EPSG:32633,4
7,"[2020-01-01, 2020-04-01]","{""type"": ""FeatureCollection"", ""features"": [{""i...",33UXA,831f0dfffffffff,EPSG:32633,3


In [20]:
import openeo
from openeo.extra.job_management import MultiBackendJobManager, CsvJobDatabase

# Authenticate and add the backend
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()

# initialize the job manager
manager = MultiBackendJobManager()
manager.add_backend("cdse", connection=connection, parallel_jobs=2)

job_tracker = 'jobs.csv'
job_db = CsvJobDatabase(path=job_tracker)
if not job_db.exists():
    df = manager._normalize_df(job_df)
    job_db.persist(df)



Authenticated using refresh token.


In [ ]:
import geojson
from s3proxy_utils import upload_geoparquet_file

def start_job(row: pd.Series, connection: openeo.Connection, **kwargs) -> openeo.BatchJob:

        temporal_extent = row["temporal_extent"]
        geom_geojson = geojson.loads(row["geometry"])

        features = gpd.GeoDataFrame.from_features(geom_geojson).set_crs(row.crs)
        url = upload_geoparquet_file(features,connection)

        s1 = connection.datacube_from_process(
                "s1_weekly_statistics",
                namespace="https://raw.githubusercontent.com/earthpulse/eotdl/refs/heads/hv_openeoexample/tutorials/notebooks/openeo/s1_weekly_statistics.json",
                temporal_extent=temporal_extent,
                )
        
        s2 = connection.datacube_from_process(
                "s2_weekly_statistics",
                namespace="https://raw.githubusercontent.com/earthpulse/eotdl/refs/heads/hv_openeoexample/tutorials/notebooks/openeo/s2_weekly_statistics.json",
                temporal_extent=temporal_extent,
                )
        
        #merged = s2.merge_cubes(s1)

        result = s1.filter_spatial(connection.load_url(url, format="Parquet"))
        result = result.save_result( format= 'NetCDF')
        
        job = result.create_job(
                sample_by_feature = True,
                feature_id_property = "file_name",
                filename_prefix = "eotdl"
        )

        return job

manager.start_job_thread(start_job=start_job, job_db=job_db)


Preflight process graph validation raised: [Internal] 'NoneType' object is not iterable
Exception in thread Thread-30:
Traceback (most recent call last):
  File "c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\requests\adapters.py", line 667, in send
    resp = conn.urlopen(
  File "c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\urllib3\connectionpool.py", line 897, in urlopen
    return self.urlopen(
  File "c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\urllib3\connectionpool.py", line 897, in urlopen
    return self.urlopen(
  File "c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\urllib3\connectionpool.py", line 897, in urlopen
    return self.urlopen(
  [Previous line repeated 2 more times]
  File "c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\urllib3\connectionpool.py", line 887, in urlopen
    retries = retri

In [ ]:
import openeo
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()
row = job_df[0:1]

geom_geojson = geojson.loads(row.geometry)
features = gpd.GeoDataFrame.from_features(geom_geojson).set_crs(row.crs)



"""
feature_crs  = row['geometry'].get("features", {})[0].get("properties", {})['crs']
features = gpd.GeoDataFrame.from_features(row['geometry']).set_crs(feature_crs)

cube = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=['2020-01-01', '2020-05-01'],
    bands=["B02"], 
    max_cloud_cover=70)

cube.filter_spatial(features)

job = cube.create_job()

job.excecute_batch()
"""

Authenticated using refresh token.
{}


'\nfeature_crs  = row[\'geometry\'].get("features", {})[0].get("properties", {})[\'crs\']\nfeatures = gpd.GeoDataFrame.from_features(row[\'geometry\']).set_crs(feature_crs)\n\ncube = connection.load_collection(\n    "SENTINEL2_L2A",\n    temporal_extent=[\'2020-01-01\', \'2020-05-01\'],\n    bands=["B02"], \n    max_cloud_cover=70)\n\ncube.filter_spatial(features)\n\njob = cube.create_job()\n\njob.excecute_batch()\n'

Step 3: set up a job manager

In [31]:
from openeo.extra.job_management import MultiBackendJobManager

# Initiate MultiBackendJobManager 
job_manager = MultiBackendJobManager()  
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()
job_manager.add_backend("cdse", connection=connection, parallel_jobs=10)

job_manager.run_jobs(job_db=job_db, start_job=job_starter)


Authenticated using refresh token.


Preflight process graph validation raised: [Internal] filter_spatial does not support '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"file_name": "SeaLake_2067", "crs": "EPSG:32633", "temporal_extent": ["2020-01-01", "2020-04-01"], "h3index": "831e02fffffffff", "tile": "33UXP"}, "geometry": {"type": "Polygon", "coordinates": [[[665860.0000000002, 5324720.0], [665860.0000000001, 5324079.999999999], [665220.0000000001, 5324079.999999998], [665220.0, 5324720.0], [665860.0000000002, 5324720.0]]]}}, {"id": "1", "type": "Feature", "properties": {"file_name": "Forest_1192", "crs": "EPSG:32633", "temporal_extent": ["2020-01-01", "2020-04-01"], "h3index": "831e06fffffffff", "tile": "33UXP"}, "geometry": {"type": "Polygon", "coordinates": [[[669840.0, 5382379.999999999], [669840.0000000001, 5381739.999999999], [669199.9999999998, 5381740.0], [669200.0000000001, 5382379.999999999], [669840.0, 5382379.999999999]]]}}, {"id": "2", "type": "Feature", "proper

RetryError: HTTPSConnectionPool(host='openeo.dataspace.copernicus.eu', port=443): Max retries exceeded with url: /openeo/1.2/jobs/j-241118f9a55147c59c9fcd3065e62f1e/results (Caused by ResponseError('too many 500 error responses'))

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Load the dataset
test = xr.open_dataset('test.nc')

# Select a specific time slice for plotting, e.g., the first time point
time_index = 1
b04 = test['B04'].isel(t=time_index)

# Create a figure to plot 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Plot each variable in a different subplot
im1 = axs[0, 0].imshow(b04, cmap='viridis', origin='lower')
axs[0, 0].set_title('B04_P10')
fig.colorbar(im1, ax=axs[0, 0])




TODO:


- Scale up (saving data, improve efficiency, ...)

